In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import seaborn as sns
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
Titles_Relabeled_2 = pd.read_csv('Titles_English_Relabeled_2.csv')

In [3]:
Titles_Relabeled_2 = Titles_Relabeled_2.drop('Unnamed: 0', axis=1)
Titles_Relabeled_2['EAN'] = Titles_Relabeled_2['EAN'].astype(str)

In [4]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1000, ngram_range=(1,2), stop_words='english')
features = tfidf.fit_transform(Titles_Relabeled_2['Annotation Text']).toarray()
labels = Titles_Relabeled_2.Category
features.shape

(300359, 3830)

In [5]:
Titles_train, Titles_test = train_test_split(Titles_Relabeled_2, random_state=42)

In [6]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=42)

# Initialize the MaxAbsScaler
scaler = MaxAbsScaler()

# Fit on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Just transform the test data
X_test_scaled = scaler.transform(X_test)

# Train the Logistic Regression model on the scaled data
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_scaled, y_train)

# Get the predicted labels and probabilities on the scaled test data
y_pred = logreg.predict(X_test_scaled)
y_pred_proba = logreg.predict_proba(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy*100:.2f}%')

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.2f}')

# Calculate recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall:.2f}')

# Create a dataframe with the predicted labels and maximum probabilities (confidence scores)
results_df = pd.DataFrame({
    'Predicted Category': y_pred,
    'Confidence Score': np.max(y_pred_proba, axis=1)
}, index=Titles_test.index)  # use the same indices as in Titles_test

# Add the 'Annotation Text' column to the results
results_df = Titles_test[['EAN', 'Title', 'Annotation Text', 'MJR BISAC']].join(results_df)

# Print the results
print(results_df)

C:\Users\Pupnsuds90\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 74.00%
F1 score: 0.76
Recall: 0.74
                    EAN                                              Title  \
272607  9780000000000.0  Dancing Bear                                  ...   
40303   9780000000000.0  Life of General Ulysses S. Grant: Containing a...   
172631  9780000000000.0  Adam's Illustrated Guide to Rye (with map): Wi...   
286961  9780000000000.0  Angela's Letter                               ...   
191935  9780000000000.0  Memoirs of a Professional Lady Nurse          ...   
...                 ...                                                ...   
182491  9780000000000.0  Feeling Special                               ...   
23941   9780000000000.0  My Father's People                            ...   
2172    9780000000000.0  Every Kid's Guide to Coping with Childhood Tra...   
114632  9780000000000.0  Nevermore...                                  ...   
153901  9780000000000.0  Sea Pictures, Op.37: Vocal score (Urtext)     ...   

            